In [59]:
# API Call =======================================================================
import requests

API_KEY = '1e3bc915825de065b5a749e727d39fcc6902538f'

year = '2018'
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F'
url = f'https://api.census.gov/data/{year}/abscs?get={get}&for=us:*&key={API_KEY}'

response = requests.get(url)
response

<Response [200]>

In [60]:
# Prepare API response for df ====================================================
# API response as a large string
data = response.text

# Remove the leading [[" and trailing "]]
data = data[3:-3]

# Split string into list. Each element is a row that will need attention
data = data.split('"],\n["')

# To split each element by the comma (while not breaking apart elements containing commas), add double quotes to null, then split on ","
new_data = [i.replace('null','"null"').split('","') for i in data]

# Check results
print(len(new_data))
for i in new_data[:3]:
    print(f'\n{i}')

19459

['GEO_ID', 'NAME', 'NAICS2017', 'NAICS2017_LABEL', 'SEX', 'SEX_LABEL', 'ETH_GROUP', 'ETH_GROUP_LABEL', 'RACE_GROUP', 'RACE_GROUP_LABEL', 'VET_GROUP', 'VET_GROUP_LABEL', 'EMPSZFI', 'EMPSZFI_LABEL', 'YEAR', 'FIRMPDEMP', 'FIRMPDEMP_F', 'RCPPDEMP', 'RCPPDEMP_F', 'EMP', 'EMP_F', 'PAYANN', 'PAYANN_F', 'FIRMPDEMP_S', 'FIRMPDEMP_S_F', 'RCPPDEMP_S', 'RCPPDEMP_S_F', 'EMP_S', 'EMP_S_F', 'PAYANN_S', 'PAYANN_S_F', 'us']

['0100000US', 'United States', '00', 'Total for all sectors', '002', 'Female', '020', 'Hispanic', '00', 'Total', '001', 'Total', '001', 'All firms', '2018', '84496', 'null', '91103725', 'null', '722654', 'null', '21975675', 'null', '3.0', 'null', '8.9', 'null', '7.7', 'null', '7.0', 'null', '1']

['0100000US', 'United States', '00', 'Total for all sectors', '002', 'Female', '020', 'Hispanic', '00', 'Total', '001', 'Total', '611', 'Firms with no employees', '2018', '10036', 'null', '3233978', 'null', '0', 'null', '499996', 'null', '4.7', 'null', '19.2', 'null', '0.0', 'null',

In [61]:
# Create a dataframe =============================================================
import pandas as pd
pd.reset_option('display.max_rows')
pd.set_option('display.max_columns', None)

data = new_data[1:]
columns = new_data[0]

census_df = pd.DataFrame(data, columns=columns)
census_df.head(3)

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F,us
0,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,001,All firms,2018,84496,null,91103725,null,722654,null,21975675,null,3.0,null,8.9,null,7.7,null,7.0,null,1
1,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,611,Firms with no employees,2018,10036,null,3233978,null,0,null,499996,null,4.7,null,19.2,null,0.0,null,13.5,null,1
2,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,612,Firms with 1 to 4 employees,2018,47105,null,15792524,null,92631,null,2808773,null,4.7,null,8.2,null,4.8,null,7.4,null,1


In [63]:
# Some mild data cleaning ========================================================
import numpy as np

census_df = pd.DataFrame(new_data[1:], columns=new_data[0])

census_df.replace(to_replace='null',value=np.NAN,inplace=True)
census_df = census_df.drop(columns=['SEX','ETH_GROUP','GEO_ID','NAME','NAICS2017','RACE_GROUP','VET_GROUP','EMPSZFI','YEAR','us'], axis=1)

census_df.head(3)

,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,EMPSZFI_LABEL,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F
0,Total for all sectors,Female,Hispanic,Total,Total,All firms,84496,NaN,91103725,NaN,722654,NaN,21975675,NaN,3.0,NaN,8.9,NaN,7.7,NaN,7.0,NaN
1,Total for all sectors,Female,Hispanic,Total,Total,Firms with no employees,10036,NaN,3233978,NaN,0,NaN,499996,NaN,4.7,NaN,19.2,NaN,0.0,NaN,13.5,NaN
2,Total for all sectors,Female,Hispanic,Total,Total,Firms with 1 to 4 employees,47105,NaN,15792524,NaN,92631,NaN,2808773,NaN,4.7,NaN,8.2,NaN,4.8,NaN,7.4,NaN


In [80]:
# As only a few lines of code ====================================================
import requests, pandas as pd, numpy as np

API_KEY = '1e3bc915825de065b5a749e727d39fcc6902538f'
year = '2018'
get = 'NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F'
url = f'https://api.census.gov/data/{year}/abscs?get={get}&for=us:*&key={API_KEY}'

# Using .TEXT ===================
# data = [i.replace('null','"null"').split('","') for i in requests.get(url).text[3:-3].split('"],\n["')]

# Using .JSON ===================
data = requests.get(url).json()

census_df = pd.DataFrame(data[1:], columns=data[0])\
    .replace(to_replace='null', value=np.NAN)\
    .drop(columns=['SEX','ETH_GROUP','NAICS2017','RACE_GROUP','VET_GROUP','EMPSZFI','us'], axis=1)

print(census_df.shape)

(19458, 25)
